[View in Colaboratory](https://colab.research.google.com/github/pgurazada/ml-projects/blob/master/2018_06_07_convnet_introduction.ipynb)

In [0]:
from keras.datasets import mnist
from keras.utils import to_categorical

In [0]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Activation
from keras import backend as K
from keras.callbacks import EarlyStopping

In [2]:
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

11493376/11490434 [==============================] - 1s 0us/step


In [0]:
train_images = train_images.reshape((60000, 28, 28, 1))
train_images = train_images.astype('float32') / 255
test_images = test_images.reshape((10000, 28, 28, 1))
test_images = test_images.astype('float32') / 255
train_labels = to_categorical(train_labels)
test_labels = to_categorical(test_labels)

In [0]:
model = Sequential()

model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)))
model.add(MaxPooling2D(2, 2))

model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(2, 2))

model.add(Conv2D(64, (3, 3), activation='relu'))

model.add(Flatten())

model.add(Dense(64, activation='relu'))

model.add(Dense(64, activation='relu'))

model.add(Dense(10, activation='softmax'))

In [52]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 26, 26, 32)        320       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 13, 13, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 11, 11, 64)        18496     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 5, 5, 64)          0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 3, 3, 64)          36928     
_________________________________________________________________
flatten_1 (Flatten)          (None, 576)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 64)                36928     
__________

In [0]:
model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])

In [54]:
%%time
training_output = model.fit(train_images, train_labels, 
                            epochs=20, 
                            batch_size=64, 
                            validation_split=0.2,
                            callbacks=[EarlyStopping(patience=2)])

Train on 48000 samples, validate on 12000 samples
Epoch 1/20
48000/48000 [==============================] - 9s 189us/step - loss: 0.2176 - acc: 0.9316 - val_loss: 0.0731 - val_acc: 0.9776
Epoch 2/20
48000/48000 [==============================] - 9s 182us/step - loss: 0.0562 - acc: 0.9828 - val_loss: 0.0497 - val_acc: 0.9844
Epoch 3/20
26176/48000 [===============>..............] - ETA: 3s - loss: 0.0371 - acc: 0.9886

48000/48000 [==============================] - 9s 184us/step - loss: 0.0381 - acc: 0.9884 - val_loss: 0.0417 - val_acc: 0.9887
Epoch 4/20
48000/48000 [==============================] - 9s 183us/step - loss: 0.0287 - acc: 0.9912 - val_loss: 0.0425 - val_acc: 0.9870
Epoch 5/20
48000/48000 [==============================] - 9s 185us/step - loss: 0.0232 - acc: 0.9927 - val_loss: 0.0426 - val_acc: 0.9878
CPU times: user 38.4 s, sys: 13 s, total: 51.4 s
Wall time: 44.6 s


In [55]:
training_output.history['val_acc']

[0.9775833333333334, 0.9844166666666667, 0.9886666666666667, 0.987, 0.98775]

In [0]:
K.clear_session()

In [1]:
!pip install kaggle


  Running setup.py bdist_wheel for kaggle ... - done
  Stored in directory: /content/.cache/pip/wheels/d6/3c/92/375decc0cf0a65b281db3cca9b8e21dc27b8ebc4d1ce29afca
Successfully built kaggle


In [0]:
from googleapiclient.discovery import build
import io, os
from googleapiclient.http import MediaIoBaseDownload
from google.colab import auth

In [0]:
auth.authenticate_user()

In [0]:
drive_service = build('drive', 'v3')
results = drive_service.files().list(
        q="name = 'kaggle.json'", fields="files(id)").execute()
kaggle_api_key = results.get('files', [])

In [0]:
filename = "/content/.kaggle/kaggle.json"
os.makedirs(os.path.dirname(filename), exist_ok=True)

In [6]:
request = drive_service.files().get_media(fileId=kaggle_api_key[0]['id'])
fh = io.FileIO(filename, 'wb')
downloader = MediaIoBaseDownload(fh, request)
done = False
while done is False:
    status, done = downloader.next_chunk()
    print("Download %d%%." % int(status.progress() * 100))
os.chmod(filename, 600)

Download 100%.


In [8]:
!kaggle competitions files -c dogs-vs-cats

name                   size  creationDate         
--------------------  -----  -------------------  
sampleSubmission.csv   87KB  2013-09-20 15:47:40  
test1.zip             271MB  2013-09-20 15:49:17  
train.zip             543MB  2013-09-20 15:52:50  


In [9]:
!kaggle competitions download  -c dogs-vs-cats -p /content/kaggle

sampleSubmission.csv: Downloaded 87KB of 87KB to /content/kaggle
test1.zip: Downloaded 271MB of 271MB to /content/kaggle
train.zip: Downloaded 543MB of 543MB to /content/kaggle


In [0]:
base_dir = '/content/kaggle/'
train_dir = os.path.join(base_dir, 'train')
os.mkdir(train_dir)
validation_dir = os.path.join(base_dir, 'validation')
os.mkdir(validation_dir)
test_dir = os.path.join(base_dir, 'test')
os.mkdir(test_dir)